In [1]:
import pandas as pd 
import re
import numpy as np
from unidecode import unidecode
import funciones
df=pd.read_excel("urbanos20-10.xlsx")
print(df.columns)

C:\Users\57317\AppData\Local\Temp\ipykernel_19688\1282955391.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Index(['CONSECUTIVO', 'ESTADO', 'Fecha de Creación', 'Cliente',
       '1.- ¿Tipo de servicio requerido? / Type of service required?', 'OTRO',
       '2.- ¿La solicitud es para mi? / Is it an own requirement?',
       '3.-Ingrese el Gin Number del pasajero / Type the Passenger Gin Number || SI ES CONTRATISTA, DEBE DIGITAR EL GIN NUMBER DEL SOLICITANTE / IF IT IS THIRD PARTY PERSON, TYPE REQUESTER GIN',
       '4.- Nombre del pasajero / Passenger´s name',
       '5.- Número del contacto / Contact Number',
       '6.- Por favor tenga en cuenta que esta cumpliendo con los siguientes requerimientos para el ingreso a la base | Please assure you have completed one site pass registration process',
       '7.- Business Line (Seleccione el BL donde pertenece / Choose the BL for the requirement)',
       '8.- Sub Business Line Reservoir Performance Evluation /RPE)',
       '9. Si seleccionaste en la pregunta 8 la opción Otro-Other, favor escribir el Sub Business Line al que perteneces / If you c

In [2]:

df=df[df["PROVEEDOR"]=="ENTRAPETROL"]
columnas=['CONSECUTIVO',"ESTADO",'PLACA VEHICULO / VEHICLE LICENSE PLATE','7.- Business Line (Seleccione el BL donde pertenece / Choose the BL for the requirement)','8.- Sub Business Line Reservoir Performance Evluation /RPE)','Cliente','16.- Seleccione el municipio de origen / Select the municipality of your origin','18.- Seleccione el municipio de destino / Select the municipality of your destination','Fecha de Creación','14.- Elija la fecha y hora del servicio / Choose service date and hour','PROVEEDOR','21.- Se requiere el viaje de retorno? / Do you need a return trip?','10.- Indique el centro de costo al que pertenece / Please type your cost center']
for col in columnas:
    if col not in df.columns:
        print(f"Column '{col}' not found in the DataFrame.")
df=df[columnas]
df.rename(columns={"CONSECUTIVO":"Order Base","PLACA VEHICULO / VEHICLE LICENSE PLATE":"Vehicle plate","16.- Seleccione el municipio de origen / Select the municipality of your origin":"Origin","18.- Seleccione el municipio de destino / Select the municipality of your destination":"Destination","14.- Elija la fecha y hora del servicio / Choose service date and hour":"Fecha y hora de cargue","FIN DE RUTA / ROUTE END":"Fecha Finalizacion","7.- Business Line (Seleccione el BL donde pertenece / Choose the BL for the requirement)":"BL","8.- Sub Business Line Reservoir Performance Evluation /RPE)":"BL2","Cliente":'Requester',"Fecha de Creación":'Fecha y hora de creacion de OB','21.- Se requiere el viaje de retorno? / Do you need a return trip?':'Retorno','10.- Indique el centro de costo al que pertenece / Please type your cost center':'WBS'},inplace=True)
df=df[df['ESTADO']!='CANCELAR']
print(df.columns)


Index(['Order Base', 'ESTADO', 'Vehicle plate', 'BL', 'BL2', 'Requester',
       'Origin', 'Destination', 'Fecha y hora de creacion de OB',
       'Fecha y hora de cargue', 'PROVEEDOR', 'Retorno', 'WBS'],
      dtype='object')


In [3]:
df['Logistic Cordinator']='Paola Andrea Chacon Acuna'
df['Shipment']=df['Order Base']
df['Period']='Jan-24'
df['Real Plate services']=df['Vehicle plate']
df['Rechargable Cost To Client']='NO'
df['Fecha Finalizacion']=df["Fecha y hora de cargue"]
# Assuming df['BL2'] contains non-string data
df['BL2'] = df['BL2'].astype(str)
# Use a regular expression to extract text within parentheses
df['BL'] = df['BL2'].apply(lambda x: re.findall(r'\((.*?)\)', x)[0] if re.findall(r'\((.*?)\)', x) else x)
# Now, you can set 'BL2' to an empty string if needed
df['BL2'] = ""
df['BL3'] = ""
df['Service Type']='RENTA FIJA OFS'
df['Service Category']='DOMESTIC'
df['Suppliers']='COL_SCHLUMBERGER'
df['Vehicle Type']='PICKUP'
df['Vehicle Type_Categoria']='LIGHT'
df['Driver']=""
df['Driver GIN']=""
df['Distance KM']='80'
df['Days of Service']='1'
df['Fecha Finalizacion'] = pd.to_datetime(df['Fecha Finalizacion'])
df['Fecha y hora de cargue'] = pd.to_datetime(df['Fecha y hora de cargue'])
df['Fecha y hora de creacion de OB'] = pd.to_datetime(df['Fecha y hora de creacion de OB'])
df['Fecha Finalizacion']+=pd.Timedelta(hours=4)
df['Freight']='160000'
desired_order=['Logistic Cordinator','Order Base','Shipment','Period','Vehicle plate','Real Plate services','Rechargable Cost To Client','BL','BL2','BL3','Service Type','Service Category','Suppliers','Vehicle Type','Vehicle Type_Categoria','Driver','Driver GIN','Requester','Origin','Destination','Distance KM','Fecha y hora de creacion de OB','Fecha y hora de cargue','Fecha Finalizacion','Freight','Days of Service','Retorno','WBS']
df=df[desired_order]
df['Fecha y hora de cargue'] = df['Fecha y hora de cargue'].dt.strftime('%m/%d/%Y %H:%M:%S')
df['Fecha y hora de creacion de OB']=df['Fecha y hora de creacion de OB'].dt.strftime('%m/%d/%Y %H:%M:%S')
df['Fecha Finalizacion']=df['Fecha Finalizacion'].dt.strftime('%m/%d/%Y %H:%M:%S')

In [4]:
def mapear_renta(row):
    rentas=pd.read_excel("Placas.xlsx",sheet_name="Renta",header=0)
    matricula=row['Vehicle plate']
    matching_row=rentas.loc[rentas['Cupo/Placa']==matricula]
    if not matching_row.empty:
        return matching_row['Clase'].values[0]
    else:
        return "No encontrado"
def provisionar(row):
    tipo=mapear_renta(row)
    if not tipo=="PICKUP":
        if tipo=="AUT HIBRIDO" or tipo=="PICK UP ELECTRICA":
            if 'Si' in row['Retorno']:
                return 350000
            else:
                return 150000         
    else:    
        if 'Si' in row['Retorno']:
            return 550000
        else:
            return 200000
def verificarWBS(cadena):
    cadena=str(cadena)
    cadena=cadena.upper()
    patron_formato1 = r'^CO\d{6}$'
    patron_formato2 = r'^J\.\d{2}\.\d{6}$'
    if re.search(patron_formato1, cadena) or re.search(patron_formato2, cadena):
        #comentado  
        return cadena
    else:
        if len(cadena)==6 and cadena.isdigit():
            cadena=f"CO{cadena}"
            return cadena
        else:
            cadena.replace(' ','')
            return cadena
df['WBS']=df['WBS'].apply(verificarWBS)        
df['Freight']=df.apply(provisionar,axis=1)

In [5]:
df.to_excel('urbanos_limpios.xlsx', engine='openpyxl', index=False)
    